In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV, train_test_split as surprise_train_test_split
from surprise import accuracy
import pyodbc
import pandas as pd
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV, train_test_split as surprise_train_test_split
from surprise import accuracy

# Configuration for SQL Server connection
server = 'YOUR_SERVER_NAME\\SQLSERVER' 
database = 'your_database_name' 

# Create the connection string for Windows Authentication
conn_str = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

# Connect to SQL Server
conn = pyodbc.connect(conn_str)

# Load datasets from SQL Server
customer_profile_query = "SELECT * FROM dim_customer"
products_query = "SELECT * FROM dim_product"
fact_query = "SELECT * FROM fact"

customer_profile = pd.read_sql(customer_profile_query, conn)
products = pd.read_sql(products_query, conn)
fact = pd.read_sql(fact_query, conn)



C:\Users\21276\AppData\Local\Temp\ipykernel_37116\310350806.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_profile = pd.read_sql(customer_profile_query, conn)
C:\Users\21276\AppData\Local\Temp\ipykernel_37116\310350806.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql(products_query, conn)
C:\Users\21276\AppData\Local\Temp\ipykernel_37116\310350806.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact = pd.read_sql(fact_query, conn)


In [4]:
customer_profile


,customer_id,first_name,last_name,gender,date_of_birth,email,phone_number,signup_date,address,city,state,zip_code
0,1,Robert,Smith,Female,1994-06-14,jane.davis1@mail.com,634-106-4981,2016-10-16,8465 Main St,San Antonio,CA,35566
1,2,Emily,Garcia,Female,1989-09-21,robert.williams2@mail.com,386-635-5998,2021-04-04,305 Main St,New York,AZ,23187
2,3,Jessica,Brown,Male,1984-01-21,emily.davis3@mail.com,627-341-5213,2018-04-22,5725 Oak St,Chicago,AZ,99188
3,4,Michael,Brown,Male,1986-02-06,jessica.williams4@mail.com,126-662-8981,2018-07-06,8468 Oak St,Los Angeles,TX,77421
4,5,Robert,Jones,Male,1996-12-05,robert.martinez5@mail.com,758-947-2802,2018-03-11,2820 Maple St,Chicago,TX,36281
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Linda,Williams,Female,1988-08-08,john.smith996@mail.com,244-837-5857,2019-05-21,2920 Pine St,New York,CA,14167
996,997,Linda,Davis,Male,1975-12-31,alex.davis997@mail.com,722-743-2177,2016-07-01,1630 Pine St,Houston,AZ,26127
997,998,Jessica,Jones,Female,1975-04-09,robert.garcia998@mail.com,186-553-4876,2018-03-22,5681 Elm St,San Antonio,PA,79596
998,999,Jane,Johnson,Male,1967-02-18,robert.johnson999@mail.com,784-272-6378,2020-03-01,8964 Oak St,New York,TX,69806


In [6]:
products


,product_id,product_name,category,price_per_unit,brand,product_description
0,1,Butter,Dairy,2858.00,BrandB,Description for Rice
1,2,Butter,Meats,2266.00,BrandB,Description for Banana
2,3,Milk,Meats,2652.00,BrandE,Description for Banana
3,4,Banana,Grains,2612.00,BrandB,Description for Apple
4,5,Rice,Fruits,2194.00,BrandD,Description for Banana
...,...,...,...,...,...,...
495,496,Cheese,Meats,20.32,BrandB,Description for Cheese
496,497,Bread,Dairy,11.53,BrandB,Description for Rice
497,498,Banana,Fruits,21.64,BrandA,Description for Rice
498,499,Chicken,Meats,22.42,BrandB,Description for Bread


In [7]:
fact

,purchase_id,customer_id,product_id,date_key,quantity,total_amount,average_amount_per_product,profit
0,1,1,42,NaN,3,3.764207e+15,1.254736e+15,3.764207e+15
1,2,1,138,20220710.0,4,7.024711e+15,1.756178e+15,7.024711e+15
2,3,1,403,20211231.0,3,8.916890e+15,2.972297e+15,8.916890e+15
3,4,1,193,NaN,2,5.970506e+15,2.985253e+15,5.970506e+15
4,5,1,26,NaN,3,1.017789e+16,3.392629e+15,1.017789e+16
...,...,...,...,...,...,...,...,...
10303,10304,999,206,20210508.0,4,7.148241e+15,1.787060e+15,7.148241e+15
10304,10305,999,393,20230227.0,2,8.434423e+15,4.217211e+15,8.434423e+15
10305,10306,999,473,20210422.0,2,8.014670e+15,4.007335e+15,8.014670e+15
10306,10307,1000,453,20230109.0,2,3.978706e+16,1.989353e+16,3.978706e+16


In [8]:
# Supprimer la colonne "product_description" 
if 'product_description' in products.columns:
    products.drop(columns=['product_description'], inplace=True)

In [9]:
# Créer une colonne "popularity" basée sur le nombre d'achats de chaque produit
product_popularity =fact['product_id'].value_counts().reset_index()
product_popularity.columns = ['product_id', 'popularity']
products = products.merge(product_popularity, on='product_id', how='left')
products['popularity'] = products['popularity'].fillna(0)

# Préparer les données pour le système de recommandation
ratings = fact[['customer_id', 'product_id', 'total_amount']]
reader = Reader(rating_scale=(ratings['total_amount'].min(), ratings['total_amount'].max()))
data = Dataset.load_from_df(ratings, reader)

In [10]:
# Recherche des hyperparamètres pour le modèle SVD
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

# Meilleurs paramètres et entraînement du modèle
print("Best RMSE score:", gs.best_score['rmse'])
print("Best parameters:", gs.best_params['rmse'])
model = gs.best_estimator['rmse']

Best RMSE score: 6.399093011852841e+16
Best parameters: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.02}


In [11]:
# Diviser les données en ensembles d'entraînement et de test
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)

# Entraîner le modèle avec les meilleurs paramètres
model.fit(trainset)



In [12]:
# Évaluer le modèle
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 64002241661493240.0000
RMSE: 6.400224166149324e+16


In [13]:
# Fonction pour recommander des produits à un utilisateur donné
def recommend_products(user_id, model, products, ratings, customer_profile, n_recommendations=5):
    product_ids = products['product_id'].unique()
    purchased_products = ratings[ratings['customer_id'] == user_id]['product_id'].unique()
    products_to_recommend = [pid for pid in product_ids if pid not in purchased_products]

    # Obtenir l'état de l'utilisateur
    user_state = customer_profile.loc[customer_profile['customer_id'] == user_id, 'state'].values[0]

    # Filtrer les produits par état
    state_products = products[products['state'] == user_state] if 'state' in products.columns else products
    predictions = [
        model.predict(user_id, pid)
        for pid in products_to_recommend if pid in state_products['product_id'].values
    ]

    # Trier les prédictions par score estimé
    predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Récupérer les meilleurs produits
    top_recommendations = predictions[:n_recommendations]
    recommended_product_ids = [pred.iid for pred in top_recommendations]
    recommended_products = products[products['product_id'].isin(recommended_product_ids)]

    return recommended_products

In [14]:
# Fonction pour recommander des produits à un nouvel utilisateur (Cold Start)
def recommend_for_new_user(birth_date, user_state, products, n_recommendations=5):
    current_year = pd.Timestamp.now().year
    user_year_of_birth = pd.to_datetime(birth_date).year
    user_age = current_year - user_year_of_birth

    # Filtrer les produits adaptés
    state_products = products[products['state'] == user_state] if 'state' in products.columns else products

    # Trier par popularité et prix pour recommander les meilleurs produits
    recommended_products = state_products.sort_values(by=["popularity", "price_per_unit"], ascending=[False, True]).head(n_recommendations)

    return recommended_products

In [15]:
# Interface utilisateur améliorée
while True:
    print("\nBienvenue dans le système de recommandation.")
    print("Si vous avez un ID utilisateur, entrez-le.")
    print("Sinon, tapez 'I am new' pour commencer.")
    print("Tapez 'quit' pour quitter.")
    
    user_input = input("\nVotre saisie : ").strip()
    
    if user_input.lower() == 'quit':
        print("Au revoir !")
        break
    
    if user_input.isdigit():  # Vérifier si l'utilisateur a saisi un ID numérique
        user_id = int(user_input)
        if user_id in ratings['customer_id'].unique():
            recommendations = recommend_products(user_id, model, products, ratings, customer_profile, n_recommendations=5)
            print(f"\nRecommandations pour l'utilisateur {user_id} :")
            print(recommendations[['product_id', 'product_name', 'price_per_unit']])
        else:
            print("\nID utilisateur non trouvé dans les données existantes.")
            print("Veuillez réessayer ou taper 'I am new' pour continuer comme nouvel utilisateur.")
    
    elif user_input.lower() == 'i am new':  # Gestion des nouveaux utilisateurs
        print("\nBienvenue ! Nous allons personnaliser vos recommandations.")
        birth_date = input("Entrez votre date de naissance (AAAA-MM-JJ) : ").strip()
        gender = input("Entrez votre sexe (Homme/Femme) : ").strip()
        user_state = input("Entrez votre state : ").strip()
        
        try:
            # Générer des recommandations pour un nouvel utilisateur
            new_user_recommendations = recommend_for_new_user(birth_date, user_state, products, n_recommendations=5)
            print("\nRecommandations pour un nouvel utilisateur :")
            print(new_user_recommendations[['product_id', 'product_name', 'price_per_unit']])
        except Exception as e:
            print("\nUne erreur est survenue. Veuillez vérifier vos informations et réessayer.")
            print(f"Erreur : {e}")
    else:
        print("\nEntrée non valide. Veuillez saisir un ID utilisateur valide, 'I am new', ou 'quit'.")


Bienvenue dans le système de recommandation.
Si vous avez un ID utilisateur, entrez-le.
Sinon, tapez 'I am new' pour commencer.
Tapez 'quit' pour quitter.

Bienvenue ! Nous allons personnaliser vos recommandations.

Recommandations pour un nouvel utilisateur :
     product_id product_name  price_per_unit
235         236       Banana            8.75
115         116       Butter         2541.00
342         343         Milk            0.43
330         331        Apple            2.37
95           96         Rice           16.25

Bienvenue dans le système de recommandation.
Si vous avez un ID utilisateur, entrez-le.
Sinon, tapez 'I am new' pour commencer.
Tapez 'quit' pour quitter.

Entrée non valide. Veuillez saisir un ID utilisateur valide, 'I am new', ou 'quit'.

Bienvenue dans le système de recommandation.
Si vous avez un ID utilisateur, entrez-le.
Sinon, tapez 'I am new' pour commencer.
Tapez 'quit' pour quitter.

Entrée non valide. Veuillez saisir un ID utilisateur valide, 'I am new